# Day 7: Some Assembly Required

[*Advent of Code 2015 day 7*](https://adventofcode.com/2015/day/7) and [*solution megathread*](https://redd.it/3vr4m4)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/07/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F07%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


In [2]:
HTML(downloaded['part1'])

In [3]:
testdata = (['123 -> x',
             '456 -> y',
             'x AND y -> d',
             'x OR y -> e',
             'x LSHIFT 2 -> f',
             'y RSHIFT 2 -> g',
             'NOT x -> h',
             'NOT y -> i'],
            {'d': 72,
             'e': 507,
             'f': 492,
             'g': 114,
             'h': 65412,
             'i': 65079,
             'x': 123,
             'y': 456})

inputdata = downloaded['input'].splitlines()

In [4]:
for data in inputdata:
    print(data)

NOT dq -> dr
kg OR kf -> kh
ep OR eo -> eq
44430 -> b
NOT gs -> gt
dd OR do -> dp
eg AND ei -> ej
y AND ae -> ag
jx AND jz -> ka
lf RSHIFT 2 -> lg
z AND aa -> ac
dy AND ej -> el
bj OR bi -> bk
kk RSHIFT 3 -> km
NOT cn -> co
gn AND gp -> gq
cq AND cs -> ct
eo LSHIFT 15 -> es
lg OR lm -> ln
dy OR ej -> ek
NOT di -> dj
1 AND fi -> fj
kf LSHIFT 15 -> kj
NOT jy -> jz
NOT ft -> fu
fs AND fu -> fv
NOT hr -> hs
ck OR cl -> cm
jp RSHIFT 5 -> js
iv OR jb -> jc
is OR it -> iu
ld OR le -> lf
NOT fc -> fd
NOT dm -> dn
bn OR by -> bz
aj AND al -> am
cd LSHIFT 15 -> ch
jp AND ka -> kc
ci OR ct -> cu
gv AND gx -> gy
de AND dk -> dm
x RSHIFT 5 -> aa
et RSHIFT 2 -> eu
x RSHIFT 1 -> aq
ia OR ig -> ih
bk LSHIFT 1 -> ce
y OR ae -> af
NOT ca -> cb
e AND f -> h
ia AND ig -> ii
ck AND cl -> cn
NOT jh -> ji
z OR aa -> ab
1 AND en -> eo
ib AND ic -> ie
NOT eh -> ei
iy AND ja -> jb
NOT bb -> bc
ha OR gz -> hb
1 AND cx -> cy
NOT ax -> ay
ev OR ew -> ex
bn RSHIFT 2 -> bo
er OR es -> et
eu OR fa -> fb
jp OR ka -> k

In [5]:
from enum import Enum, auto

class Gate(Enum):
    INPUTVALUE = 'INPUTVALUE'
    NOT = 'NOT'
    LSHIFT = 'LSHIFT'
    RSHIFT = 'RSHIFT'
    AND = 'AND'
    OR = 'OR'

    def __str__(self):
        return self.value

class CircuitPart:
    outwire = ""
    p1 = p2 = ""
    gate = None

    def __init__(self, circuitstr: str):
        split = circuitstr.split()
        self.outwire = split[-1]
        if len(split) == 3:
            self.gate = Gate.INPUTVALUE
            self.p1 = split[0]
        elif len(split) == 4:
            self.gate = Gate.NOT
            self.p1 = split[1]
        elif len(split) == 5:
            self.gate = Gate[split[1]]
            self.p1 = split[0]
            self.p2 = split[2]

    def __str__(self):
        left = ""
        if self.gate == Gate.INPUTVALUE:
            left = self.p1
        elif self.gate == Gate.NOT:
            left = f'NOT {self.p1}'
        else:
            left = f'{self.p1} {self.gate} {self.p2}'
        
        return(f'{left} -> {self.outwire}')

    def __repr__(self):
        return self.__str__()

In [6]:
for circuitstr in testdata[0]:
    print(CircuitPart(circuitstr))

123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i


In [7]:
import numpy as np

wires = {}

def evaluate_part(part: CircuitPart, wire_b_to_a=None) -> bool:
    if part.p1 in wires.keys():
        p1 = wires[part.p1]
    else:
        try:
            p1 = np.uint16(part.p1)
        except ValueError:  # The argument was not a known wire but also not an integer
            return False
    
    if part.gate == Gate.INPUTVALUE:
        if wire_b_to_a and part.outwire == 'b':
            wires['b'] = wire_b_to_a
            return True

        wires[part.outwire] = p1
        return True
    elif part.gate == Gate.NOT:
        wires[part.outwire] = np.invert(p1)
        return True
    
    if part.p2 in wires.keys():
        p2 = wires[part.p2]
    else:
        try:
            p2 = np.uint16(part.p2)
        except ValueError:
            return False
    
    if part.gate == Gate.LSHIFT:
        wires[part.outwire] = np.left_shift(p1, p2)
        return True
    elif part.gate == Gate.RSHIFT:
        wires[part.outwire] = np.right_shift(p1, p2)
        return True
    elif part.gate == Gate.OR:
        wires[part.outwire] = np.bitwise_or(p1, p2)
        return True
    elif part.gate == Gate.AND:
        wires[part.outwire] = np.bitwise_and(p1, p2)
        return True
    else:
        raise ValueError(f'Could not apply {part} on {wires}')


def evaluate_circuit(circuitparts, wire_b_to_a=None):
    remainingparts = list(circuitparts)[:]
    while len(remainingparts) > 0:
        still_remaining = []
        for part in remainingparts:
            if not evaluate_part(part, wire_b_to_a=wire_b_to_a):
                still_remaining.append(part)
            # else:
            #     print(f'Successfully evaluated {part} for {wires}')

        if len(still_remaining) == len(remainingparts):
            print(f'Failed to evaluate any remaining circuit parts: {still_remaining} for {wires}')
            return
        
        remainingparts = still_remaining

In [8]:
wires = {}
evaluate_circuit(map(CircuitPart,testdata[0]))
print(wires)

{'x': 123, 'y': 456, 'd': 72, 'e': 507, 'f': 492, 'g': 114, 'h': 65412, 'i': 65079}


In [9]:
wires = {}
circuitparts = list(map(CircuitPart, inputdata))
evaluate_circuit(circuitparts)
print(wires['a'])

3176


In [10]:
HTML(downloaded['part1_footer'])

In [11]:
HTML(downloaded['part2'])

In [12]:
b = wires['a']
wires = {}
evaluate_circuit(circuitparts, wire_b_to_a=b)
print(wires['a'])

14710


In [13]:
HTML(downloaded['part2_footer'])